In [85]:
from adapters import AutoAdapterModel
from transformers import AutoTokenizer, AutoConfig
from datasets import load_dataset
from torch.utils.data import Dataset
import torch
import torch.nn.functional as F
from tqdm.notebook import tqdm
from torch import nn
import adapters.composition as ac
from transformers import AutoConfig
from adapters import AutoAdapterModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import evaluate
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import adapters.composition as ac
from transformers import AutoConfig
from adapters import AutoAdapterModel
import adapters.composition as ac
from transformers import AutoConfig
from adapters import AutoAdapterModel

In [86]:

def compute_metrics(eval_pred, task_name):
    # print(eval_pred)
    logits, labels = eval_pred
    if task_name == 'ast':
      predictions = np.argmax(logits, axis=-1)
      accuracy = accuracy_score(labels, predictions)
      precision = precision_score(labels, predictions, average='weighted')
      recall = recall_score(labels, predictions, average='weighted')
      f1 = f1_score(labels, predictions, average='weighted')

      return {
          'accuracy': accuracy,
          'precision': precision,
          'recall': recall,
          'f1': f1,
      }
    elif task_name == 'ner':
      seqeval = evaluate.load("seqeval")
      predictions = np.argmax(logits, axis=2)
      label_list  = ["O", "B-test", "I-test", "B-problem", "I-problem", "B-treatment", "I-treatment"]
      true_predictions = [
          [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
          for prediction, label in zip(predictions, labels)
      ]
      true_labels = [
          [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
          for prediction, label in zip(predictions, labels)
      ]

      results = seqeval.compute(predictions=true_predictions, references=true_labels)
      return {
          "accuracy": results["overall_accuracy"],
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"]
      }

### Load BERT and add adapters

In [88]:
# pretrained_model_name_or_path = "bert-base-uncased"
# #pretrained_model_name_or_path = "emilyalsentzer/Bio_Discharge_Summary_BERT"
# tokenizer  = AutoTokenizer.from_pretrained(pretrained_model_name_or_path, model_max_length=150)
# special_tokens_dict = {"additional_special_tokens": ["[entity]"]}
# num_added_toks = tokenizer.add_special_tokens(special_tokens_dict,False)
# model = AutoAdapterModel.from_pretrained(pretrained_model_name_or_path = pretrained_model_name_or_path,
#                                          num_labels=3 , id2label = {0: 'PRESENT', 1: 'ABSENT', 2:'POSSIBLE'})
# model.resize_token_embeddings(len(tokenizer))
# # ast = model.load_adapter("./adapter_ast_SeqBnConfig_bert/",with_head=True)
# ast = model.load_adapter(r"adapter-fusion-weights\bert\ast\adapter_ast_SeqBnConfig_clinicalbert",with_head=True)

# #ast = model.load_adapter("./adapter_ast_pfeiffer_clinicalbert",with_head=True)
# model.active_adapters = ast
# num_params = sum(p.numel() for p in model.base_model.parameters() if p.requires_grad )
# print()
# print(f"Number of trainable parameters: {num_params}")
# print()
# # How you can acces the labels and the mapping for a pretrained head
# print(model.get_labels())
# print(model.get_labels_dict())

### Load BERT or Clinical BERT fine-tuned models

In [99]:
# FULL FINE-TUNE BERT
pretrained_model_name_or_path = r"fine-tuned_bert\ast_full"
tokenizer_path = "bert-base-uncased"

# FULL FINE-TUNE ClinicalBERT
# pretrained_model_name_or_path = r"fine-tuned_clinicalbert\ast_full"
# tokenizer_path = "emilyalsentzer/Bio_Discharge_Summary_BERT"


tokenizer  = AutoTokenizer.from_pretrained(tokenizer_path, model_max_length=150)
special_tokens_dict = {"additional_special_tokens": ["[entity]"]}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict,False)
model = AutoAdapterModel.from_pretrained(pretrained_model_name_or_path = pretrained_model_name_or_path,
                                         num_labels=3 , id2label = {0: 'PRESENT', 1: 'ABSENT', 2:'POSSIBLE'})
model.resize_token_embeddings(len(tokenizer))
num_params = sum(p.numel() for p in model.base_model.parameters() if p.requires_grad )
print()
print(f"Number of trainable parameters: {num_params}")
print()
# How you can acces the labels and the mapping for a pretrained head
print(model.get_labels())
print(model.get_labels_dict())


Number of trainable parameters: 109483008

['LABEL_0', 'LABEL_1', 'LABEL_2']
{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}


In [95]:
from transformers import TrainingArguments, EvalPrediction
from utils import AssertionDatai2b2, ConceptDatai2b2
from datasets import Dataset, DatasetDict
from adapters import AdapterTrainer
from transformers import TrainingArguments, Trainer

preprocessed_data_path = "Data/preprocessed-data"
train_data_path = "Data/concept_assertion_relation_training_data"
reference_test_data_path = "Data/reference_standard_for_test_data"
test_data_path = "Data/test_data"
ast_i2b2 = AssertionDatai2b2(preprocessed_data_path=preprocessed_data_path,
                                 train_data_path=train_data_path,
                                 reference_test_data_path=reference_test_data_path,
                                 test_data_path=test_data_path)
beth_and_partners_data, test_data, all_data = ast_i2b2.load_assertion_i2b2_data()
# https://gist.github.com/vincenttzc/ceaa4aca25e53cb8da195f07e7d0af92


def tokenize_function_ast(example):
    return tokenizer(example["new_line"],   padding="max_length", truncation=True)
lbl2id ={'absent': 1 ,'possible': 2, 'present':0}
beth_and_partners_data['label_ids']= beth_and_partners_data.apply(lambda x: lbl2id[x['label']],axis=1)
ds_train = Dataset.from_pandas(beth_and_partners_data[['label_ids','new_line']])

tokenized_ds_train = ds_train.map(tokenize_function_ast)
tokenized_ds_train.set_format("torch")

# trainer = AdapterTrainer(model,compute_metrics=lambda p: compute_metrics(p, task_name='ast'))
# trainer = Trainer(model,compute_metrics=lambda p: compute_metrics(p, task_name='ast'))
trainer = Trainer(model, compute_metrics=compute_metrics)
outputs = trainer.predict(tokenized_ds_train)
print(outputs.metrics)


Loading data...

number of beth training records: 73
number of partners training records: 97
number of all test records: 256
total number of all combined records: 426

no labels for record-58
no labels for 262182942
no labels for 0305

number of beth records with labels: 72
number of partners records with labels: 96
number of test records with labels: 255
total number of all combined records with labels: 423

number of beth_and_partners examples: 6529
number of test examples: 11868
total number of combined examples: 18397



Map:   0%|          | 0/6529 [00:00<?, ? examples/s]

{'test_runtime': 86.327, 'test_samples_per_second': 75.631, 'test_steps_per_second': 9.464}


In [96]:
y_pred = np.argmax(outputs[0][1], axis=1)
y_true = beth_and_partners_data['label_ids']

# Micro F1 Score
micro_f1 = f1_score(y_true, y_pred, average='micro')

# Macro F1 Score
macro_f1 = f1_score(y_true, y_pred, average='macro')
print("TRAINING DATA SCORES:")
print(f"Micro F1 Score: {micro_f1}")
print(f"Macro F1 Score: {macro_f1}")

class_f1_scores = f1_score(y_true, y_pred, average=None)

# Print F1 scores for each class
for class_idx, f1_score_value in enumerate(class_f1_scores):
    print(f"Class {class_idx}: F1 Score = {f1_score_value}")

TRAINING DATA SCORES:
Micro F1 Score: 0.9932608362689539
Macro F1 Score: 0.9833025198050361
Class 0: F1 Score = 0.9958945548833189
Class 1: F1 Score = 0.9915333960489182
Class 2: F1 Score = 0.9624796084828712


In [97]:
test_data['label_ids']= test_data.apply(lambda x: lbl2id[x['label']],axis=1)
ds_test = Dataset.from_pandas(test_data[['label_ids','new_line']])

tokenized_ds_test = ds_test.map(tokenize_function_ast)
tokenized_ds_test.set_format("torch")

# trainer = AdapterTrainer(model,compute_metrics=lambda p: compute_metrics(p, task_name='ast'))
test_trainer = Trainer(model,compute_metrics=lambda p: compute_metrics(p, task_name='ast'))
test_outputs = test_trainer.predict(tokenized_ds_test)
test_outputs.metrics

Map:   0%|          | 0/11868 [00:00<?, ? examples/s]

{'test_runtime': 176.9422,
 'test_samples_per_second': 67.073,
 'test_steps_per_second': 8.387}

In [100]:
y_pred_test = np.argmax(test_outputs[0][1], axis=1)
y_true_test = test_data['label_ids']

# Micro F1 Score
micro_f1_test = f1_score(y_true_test, y_pred_test, average='micro')

# Macro F1 Score
macro_f1_test = f1_score(y_true_test, y_pred_test, average='macro')
print("TEST DATA SCORES:")
print(f"Micro F1 Score: {micro_f1_test}")
print(f"Macro F1 Score: {macro_f1_test}")

class_f1_scores_test = f1_score(y_true_test, y_pred_test, average=None)

# Print F1 scores for each class
for class_idx, f1_score_value in enumerate(class_f1_scores_test):
    print(f"Class {class_idx}: F1 Score = {f1_score_value}") #{'absent': 1 ,'possible': 2, 'present':0}

TEST DATA SCORES:
Micro F1 Score: 0.9630097741826761
Macro F1 Score: 0.8991750228843692
Class 0: F1 Score = 0.9763071424453795
Class 1: F1 Score = 0.9668465690053971
Class 2: F1 Score = 0.7543713572023313


## Test single example prediction

In [101]:
id2label = {0: 'PRESENT', 1: 'ABSENT', 2:'POSSIBLE'}
sentence = [ "Patient denies [entity] SOB [entity]", #ABSENT
            "Patient do not have [entity] fever [entity]", #ABSENT
            "had [entity] abnormal ett [entity] and referred for cath", #PRESENT
            "The patient recovered during the night and now denies any [entity] shortness of breath [entity].", #ABSENT
            "Patient with [entity] severe fever [entity].", #PRESENT
            "Patient should abstain from [entity] painkillers [entity]"] #ABSENT
model.to('cpu')
for s in sentence :
  tokenized_input = tokenizer(s, return_tensors="pt", padding=True)
  outputs = model(**tokenized_input)
  predicted_labels = torch.argmax(outputs.logits, dim=1)
  print(id2label[predicted_labels.item()])

ABSENT
ABSENT
PRESENT
ABSENT
PRESENT
ABSENT


In [102]:
# Made up examples intended to trick model
id2label = {0: 'PRESENT', 1: 'ABSENT', 2:'POSSIBLE'}
sentence = [ "Patient denies that the have experienced any type of [entity] back pain [entity]", #ABSENT
            "There was scant evidence of [entity] fever [entity]", #ABSENT
            "Patient mentioned possibility of [entity] concussion [entity] but x-ray ruled it out", #ABSENT
            "The patient recovered during the night and now denies any [entity] shortness of breath [entity].", #ABSENT
            "Patient reported [entity] severe fever [entity] but testing showed no severe fever.", #ABSENT
            "Patient should abstain from [entity] painkillers [entity]"] #ABSENT
model.to('cpu')
for s in sentence :
  tokenized_input = tokenizer(s, return_tensors="pt", padding=True)
  outputs = model(**tokenized_input)
  predicted_labels = torch.argmax(outputs.logits, dim=1)
  print(id2label[predicted_labels.item()])

ABSENT
ABSENT
POSSIBLE
ABSENT
PRESENT
ABSENT


In [103]:
for name, param in model.named_parameters():
    print(name, param.data.shape)

bert.embeddings.word_embeddings.weight torch.Size([30523, 768])
bert.embeddings.position_embeddings.weight torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight torch.Size([2, 768])
bert.embeddings.LayerNorm.weight torch.Size([768])
bert.embeddings.LayerNorm.bias torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder